In [2]:
# import preprocess as prep
import model

# preprocess / load preprocessed data (preprocess.py)
# mini batching

# load model(can quickly change model by changing imported model)
# train / log
# generate checkpoints

In [4]:
model.train(1,1,1)

IndexError: tuple index out of range